In [2]:
import os
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 문서 경로 설정
MD_DIR = "../backend/documents/md"

# pyzerox 파일을 제외한 5개 문서 파일 목록
md_files = [
    "국회본회의_회의록._제429회(14차).md",
    "국회본회의_회의록._제429회(15차).md",
    "국회본회의_회의록._제429회(16차).md",
    "국회본회의_회의록._제430회(1차).md",
    "국회본회의_회의록._제430회(2차).md"
]

# Text Splitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", "○"] 
)

def extract_doc_metadata(filename: str) -> dict:
    """파일명에서 문서 메타데이터 추출"""
    pattern = r"제(\d+)회\((\d+)차\)"
    match = re.search(pattern, filename)
    
    session = match.group(1) if match else "unknown"
    meeting = match.group(2) if match else "unknown"
    
    return {
        "source": filename,
        "session": session,
        "meeting": meeting,
        "doc_type": "국회본회의_회의록"
    }

def create_document_header(metadata: dict) -> str:
    """청크 헤더 생성"""
    header = f"""[문서 정보]
- 문서명: {metadata['source']}
- 회기: 제{metadata['session']}회
- 차수: {metadata['meeting']}차
- 유형: {metadata['doc_type']}
---
"""
    return header

def process_all_documents() -> list:
    """모든 문서를 처리하여 하나의 document_list로 반환"""
    all_documents = []
    
    for md_file in md_files:
        file_path = os.path.join(MD_DIR, md_file)
        
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        doc_metadata = extract_doc_metadata(md_file)
        header = create_document_header(doc_metadata)
        chunks = text_splitter.split_text(content)
        
        for i, chunk in enumerate(chunks):
            chunk_with_header = header + chunk
            doc = Document(
                page_content=chunk_with_header,
                metadata={
                    **doc_metadata,
                    "chunk_index": i,
                    "total_chunks": len(chunks)
                }
            )
            all_documents.append(doc)
        
        print(f"처리 완료: {md_file} - {len(chunks)}개 청크 생성")
    
    print(f"\n총 {len(all_documents)}개 Document 생성")
    return all_documents

# 실행
document_list = process_all_documents()

처리 완료: 국회본회의_회의록._제429회(14차).md - 433개 청크 생성
처리 완료: 국회본회의_회의록._제429회(15차).md - 47개 청크 생성
처리 완료: 국회본회의_회의록._제429회(16차).md - 215개 청크 생성
처리 완료: 국회본회의_회의록._제430회(1차).md - 1827개 청크 생성
처리 완료: 국회본회의_회의록._제430회(2차).md - 997개 청크 생성

총 3519개 Document 생성


In [ ]:
document_list

In [4]:
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

embeddings = UpstageEmbeddings(model = "solar-embedding-1-large-query")

vector_store = Chroma.from_documents(
    documents=document_list,
    collection_name="parliament_records",
    embedding=embeddings,
    persist_directory="../backend/vector_store"
)

In [5]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [6]:
query = "한지아 의원'이 대표발의한 법안의 내용은?"

In [7]:
retriever.invoke(query)

[Document(id='d8562dee-762a-44c7-8567-af500ea3b517', metadata={'source': '국회본회의_회의록._제429회(14차).md', 'total_chunks': 433, 'chunk_index': 420, 'session': '429', 'doc_type': '국회본회의_회의록', 'meeting': '14'}, page_content='[문서 정보]\n- 문서명: 국회본회의_회의록._제429회(14차).md\n- 회기: 제429회\n- 차수: 14차\n- 유형: 국회본회의_회의록\n---\n---\n*174 제429회-제14차(2025년12월2일)*\n\n\n## (2025. 11. 4. 김원이 의원 대표발의)(의안번호 2213909)\n\n\n\n## 산업 디지털 전환 촉진법 일부개정법률안\n\n\n(2025. 8. 6. 김원이 의원 대표발의)(의안번호 2212017)\n\n\n산업 디지털 전환 촉진법 일부개정법률안\n\n\n(2025. 8. 11. 박성민 의원 대표발의)(의안번호 2212087)\n\n\n폐광지역 개발 지원에 관한 특별법 일부개정법률안\n\n\n(2025. 10. 1. 이철규 의원 대표발의)(의안번호 2213413)\n\n\n폐광지역 개발 지원에 관한 특별법 일부개정법률안\n\n\n(2025. 10. 16. 이철규 의원 대표발의)(의안번호 2213622)\n\n\n(이상 33건 본회의에 부의하지 아니하고 이에 대한 대안 제출)\n이상 40건 산업통상자원중소벤처기업위원장 보고\n\n\n\n장애인복지법 일부개정법률안\n\n\n(2024. 9. 23. 이수진 의원 대표발의)(의안번호 2204168)\n\n\n아동복지법 일부개정법률안\n\n\n(2024. 8. 27. 김미애 의원 대표발의)(의안번호 2203223)\n\n\n의료기사 등에 관한 법률 일부개정법률안\n\n\n(2025. 2. 27. 남인순 의원 대표발의)(의안번호 2208530)\n\n\n검역법 일부개정법률안\n\n\n(2025. 3.

In [9]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str


In [10]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)


In [11]:
def retriece(state: AgentState) -> AgentState:
    query = state['query']
    docs = retriever.invoke(query)
    
    return {'context': docs}


In [ ]:
from langchain_community import hub
from langchain_upstage import UpstageChatOpenAI
prompt = hub.pull("rlm/rag-prompt")
llm = UpstageChatOpenAI

ImportError: cannot import name 'hub' from 'langchain' (/Users/parkjehyeong/jungjung/.venv/lib/python3.11/site-packages/langchain/__init__.py)